In [1]:
import pandas as pd
import numpy as np
import pickle
import gc
import os
import time
import copy
import multiprocessing as mp
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from scipy import sparse, spatial
import warnings
warnings.filterwarnings('ignore')
tqdm.pandas()

In [2]:
tic = time.time()
DATA_PATH = '../pkl/'
SAVE_PATH = './feats/'
if not os.path.exists(SAVE_PATH):
    print('create dir: %s' % SAVE_PATH)
    os.mkdir(SAVE_PATH)

print('gen_prev_ans_ques_set...')

gen_prev_ans_ques_set...


In [8]:
answer_info = pd.read_pickle(os.path.join(DATA_PATH, 'answer_info_tiny.pkl'))
inv_last_answer_time = pd.read_pickle(os.path.join(DATA_PATH, 'inv_last_answer_time.pkl'))

In [5]:
answer_info['user_time'] = answer_info['uid'].astype(str) + '_' + answer_info['atime'].astype(str)
answer_info.sort_values(by=['uid', 'user_time'],inplace=True)
answer_info = answer_info[['qid', 'uid', 'user_time']].reset_index(drop=True)

In [6]:
prev_ans_ques = []
ques = []
last = None
for a, q in tqdm(answer_info[['uid', 'qid']].values):
    if last is None or last != a:
        ques = [q]
    else:
        ques.append(q)
    prev_ans_ques.append(list(ques))
    last = a

100%|██████████| 4513735/4513735 [00:18<00:00, 240827.44it/s]


In [7]:
answer_info['prev_ans_ques'] = prev_ans_ques
del answer_info['qid'], answer_info['uid']
answer_info.drop_duplicates(subset='user_time', keep='last', inplace=True)

In [10]:
tmp = inv_last_answer_time.merge(answer_info, 'left', 'user_time')

In [13]:
tmp[['prev_ans_ques']].to_pickle(os.path.join(DATA_PATH, 'prev_ans_ques.pkl'))
tmp.head(5)

,uid,id,time,last_time,user_time,invite_answer_gap,prev_ans_ques
0,401693808,0.0,92782,NaN,401693808_-1,NaN,NaN
1,3392373099,1.0,92267,NaN,3392373099_-1,NaN,NaN
2,2317670257,2.0,92703,92539.0,2317670257_92539,164.0,"[3034061280, 1309562090]"
3,1618461867,3.0,92387,NaN,1618461867_-1,NaN,NaN
4,3544409350,4.0,92812,NaN,3544409350_-1,NaN,NaN


In [12]:
toc = time.time()
print('Used time: %d' % int(toc-tic))

Used time: 346
